# FASE 0. CREACIÓN DE ENTORNO VIRTUAL E INICIALIZACIÓN

In [ ]:
# CREACION DE ENTORNO VIRTUAL 

##Creación del entorno
    #py -3.10 -m venv rag_chatbot_py10
  

## Inicialización del entorno
    #rag_chatbot_py10\Scripts\activate


#COMPROBAR QUE SE ESTÁ EN EL ENTORNO VIRTUAL Y EN LA CARPETA DEL ENTORNO VIRTUAL




# INSTALACIÓN DE LIBRERÍAS

In [ ]:

#INSTALACIÓN DE LIBRERIAS

# python.exe -m pip install --upgrade pip

# pip install ipykernel
#python -m ipykernel install --user --name=rag_chatbot-env ///Installed kernelspec bolivar_prueba-env in C:\Users\smart\AppData\Roaming\jupyter\kernels\bolivar_prueba-env


# pip install langchain openai faiss-cpu pypdf tiktoken python-dotenv streamlit sentence_transformers pdfplumber openai accelerate nltk spacy requests

#python -m spacy download en_core_web_sm
#python -m spacy download es_core_news_sm 


# FUNCIÓN ingest_document
Se crea la función con el objetivo de convertir el documento PDF o TXT a una cadena de texto o string única.

In [ ]:
import os
from typing import List, Dict
import pdfplumber
import re

# --- CONFIGURACIÓN PARAMETRIZABLE ---
EMBED_MODEL = "deepseek-ai/DeepSeek-R1"       # o SBERT, etc.
CHUNK_SIZE = 1000            # caracteres por chunk
CHUNK_OVERLAP = 200          # solapamiento
INDEX_PATH = "faiss_index"   # carpeta para persistir índice
# -------------------------------------

def ingest_document(path: str) -> str:

    text = ""
    if path.lower().endswith(".pdf"):
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
    elif path.lower().endswith(".txt"):
        with open(path, encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError("Formato no soportado")
    return text



Se asigna la ubicación, bien sea local o en nube, del documento a analizar en "path1".

In [2]:
path1="D:/CHECHO/Codigo_Prueba_SegurosBolivar/GallagherRe Global Insurtech Report 2024-Q4.pdf"

Asignamos a texto_cargado la función con el fin de visualizar cómo se cargaron los datos.

In [ ]:
texto_cargado = ingest_document(path1)

In [ ]:
texto_cargado

# CREACIÓN DE LA FUNCIÓN clean_text
Este bloque limpia caracteres extraños y corrige errores tipográficos.

In [ ]:
def clean_text(text: str) -> str:
    
    text = re.sub(r'\s+', ' ', text)                   # espacios múltiples
    text = re.sub(r'[^ -~]', '', text)                 # caracteres no ASCII
    return text.strip()

In [5]:
texto_limpio = clean_text(texto_cargado)

In [6]:
texto_limpio

'Global FEBRUARY 2025 Q4 InsurTech Report The role of AI in the (re)insurance industryThis report is a collaboration between Gallagher Re, Gallagher, and CB Insights. 2Contents INSIDE THIS EDITION... 4. 26. 32. Introduction InsurTech Team Corner InsurTech Case Studies Building, Buying or Renting Technology Q4 InsurTech  Nearmap investment data highlights and  EvolutionIQ foreword  Pindrop 40. 46. 52. Deal of the Quarter Partnership Incumbent Agentech Case Study Corner Loadsure and Concirrus and Google Cloud United Risk 58. 64. Gallagher\'s Vision Thought Leadership Joe Powell, Gallagher Bassett Christina Lucas, Guidewire 68. 74. Investor Corner The Data Center Matthew Jones, Transverse Ventures This quarters data highlights 3Introduction Q4 InsurTech investment data highlights and foreword 4Introduction DR. ANDREW JOHNSTON Global Head of Gallagher Re InsurTech, Global InsurTech Report Editor This report is the fourth and final of our 2024 series: the role of AI in the (re)insurance val

# CREACIÓN DE CHUNKS
Debido a que sería casi imposible para un sistema analizar toda la información en un único string,
se procede con la generación de chunks o divisiones de texto, los cuales estarán limitadas por la cantidad de caracteres por chunk, en este caso 1000.

In [6]:
import spacy

# Carga el modelo para español (o inglés)
nlp = spacy.load("en_core_web_sm")

def chunk_text_spacy(text: str, max_chars: int = 1000, overlap: int = 1) -> List[Dict]:
    # Procesa todo el texto
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    
    chunks, current, length = [], [], 0
    for sent in sentences:
        slen = len(sent)
        if length + slen > max_chars and current:
            chunks.append({"chunk_id": len(chunks), "text": " ".join(current)})
            # superpone últimas 'overlap' oraciones
            current = current[-overlap:] if overlap > 0 else []
            length = sum(len(s) for s in current)
        current.append(sent)
        length += slen
    if current:
        chunks.append({"chunk_id": len(chunks), "text": " ".join(current)})
    return chunks


In [7]:
texto_chunkeado = chunk_text_spacy(texto_limpio)

In [9]:
texto_chunkeado

[{'chunk_id': 0,
  'text': "Global FEBRUARY 2025 Q4 InsurTech Report The role of AI in the (re)insurance industryThis report is a collaboration between Gallagher Re, Gallagher, and CB Insights. 2Contents INSIDE THIS EDITION... 4. 26. 32. Introduction InsurTech Team Corner InsurTech Case Studies Building, Buying or Renting Technology Q4 InsurTech  Nearmap investment data highlights and  EvolutionIQ foreword  Pindrop 40. 46. 52. Deal of the Quarter Partnership Incumbent Agentech Case Study Corner Loadsure and Concirrus and Google Cloud United Risk 58. 64. Gallagher's Vision Thought Leadership Joe Powell, Gallagher Bassett Christina Lucas, Guidewire 68. 74. Investor Corner The Data Center Matthew Jones, Transverse Ventures This quarters data highlights 3Introduction Q4 InsurTech investment data highlights and foreword 4Introduction DR. ANDREW JOHNSTON Global Head of Gallagher Re InsurTech, Global InsurTech Report Editor This report is the fourth and final of our 2024 series: the role of A

# COMPROBACIÓN DE CANTIDAD DE TOKENS(CARÁCTERES) POR CHUNK

Esta comprobación se realiza con la finalidad de comprobar que ningún chunk tenga más de la cantidad de carácteres permitidos por los LLM a utilizar, en este caso 'mistral'. 

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

for chunk in texto_chunkeado:
    text = chunk["text"]
    tokens = tokenizer(text, return_tensors="pt",truncation= True)
    num_tokens = tokens.input_ids.shape[1]
    print(f"Chunk ID {chunk['chunk_id']} tiene {num_tokens} tokens")





d:\CHECHO\Personal\PROYECTOS PERSONALES PORTAFOLIO\PROYECTO RAG CHATBOT GENERICO\rag_chatbot_py10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Chunk ID 0 tiene 222 tokens
Chunk ID 1 tiene 202 tokens
Chunk ID 2 tiene 214 tokens
Chunk ID 3 tiene 233 tokens
Chunk ID 4 tiene 245 tokens
Chunk ID 5 tiene 231 tokens
Chunk ID 6 tiene 165 tokens
Chunk ID 7 tiene 215 tokens
Chunk ID 8 tiene 241 tokens
Chunk ID 9 tiene 285 tokens
Chunk ID 10 tiene 241 tokens
Chunk ID 11 tiene 281 tokens
Chunk ID 12 tiene 388 tokens
Chunk ID 13 tiene 243 tokens
Chunk ID 14 tiene 253 tokens
Chunk ID 15 tiene 207 tokens
Chunk ID 16 tiene 190 tokens
Chunk ID 17 tiene 317 tokens
Chunk ID 18 tiene 292 tokens
Chunk ID 19 tiene 217 tokens
Chunk ID 20 tiene 189 tokens
Chunk ID 21 tiene 201 tokens
Chunk ID 22 tiene 185 tokens
Chunk ID 23 tiene 198 tokens
Chunk ID 24 tiene 155 tokens
Chunk ID 25 tiene 173 tokens
Chunk ID 26 tiene 171 tokens
Chunk ID 27 tiene 168 tokens
Chunk ID 28 tiene 177 tokens
Chunk ID 29 tiene 191 tokens
Chunk ID 30 tiene 161 tokens
Chunk ID 31 tiene 168 tokens
Chunk ID 32 tiene 185 tokens
Chunk ID 33 tiene 166 tokens
Chunk ID 34 tiene 157 to

# ALMACENAMIENTO VECTORIAL
Este bloque de código crea un índice vectorial a partir de un conjunto de textos previamente divididos en chunks.

 Utiliza el modelo de sentence transformers "all-MiniLM-L6-v2" para generar embeddings de cada fragmento y luego los almacena en un índice FAISS para realizar búsquedas semánticas rápidas. También guarda un archivo de metadatos que asocia cada embedding con su texto original.

In [9]:
from sentence_transformers import SentenceTransformer
import faiss
import os
import pickle
# ============ CONFIG ==============
EMBED_MODEL_NAME = "all-MiniLM-L6-v2"
INDEX_PATH = "vectorstore/index.faiss"
METADATA_PATH = "vectorstore/metadata.pkl"
CHUNKS = texto_chunkeado 

model = SentenceTransformer(EMBED_MODEL_NAME)
texts = [chunk['text'] for chunk in texto_chunkeado]
embeddings = model.encode(texts, show_progress_bar=True)

# Paso 2: Indexar en FAISS
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Guardar FAISS index
os.makedirs("vectorstore", exist_ok=True)
faiss.write_index(index, INDEX_PATH)

# Guardar metadata
with open(METADATA_PATH, "wb") as f:
    pickle.dump(CHUNKS, f)

print("✅ Indexación completada.")


Batches:   0%|          | 0/12 [00:00<?, ?it/s]d:\CHECHO\Personal\PROYECTOS PERSONALES PORTAFOLIO\PROYECTO RAG CHATBOT GENERICO\rag_chatbot_py10\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 12/12 [00:10<00:00,  1.11it/s]


✅ Indexación completada.


In [10]:
def search_similar(query, k=3):
    # Cargar todo
    index = faiss.read_index(INDEX_PATH)
    with open(METADATA_PATH, "rb") as f:
        metadata = pickle.load(f)
    
    # Embedding de la pregunta
    query_embedding = model.encode([query])

    # Buscar k chunks más parecidos
    distances, indices = index.search(query_embedding, k)
    
    results = []
    for idx in indices[0]:
        results.append(metadata[idx]['text'])

    return results



In [12]:
query = "What is the function of Google Cloud in the document?"
results = search_similar(query)

print("🔍 Context retrieved:")
for i, r in enumerate(results):
    print(f"{i+1}. {r}\n")

🔍 Context retrieved:
1. approach increases recovery ratios while enhancing the policyholder experience by Miles Thorson expediting deductible returns. CEO and Co-Founder of Odie Lyle Donan Co-Founder and Chairman of Source7 4,000 3,000 2,000 1,000 44 raeY reP smialC egarevA Deal of the Quarter Annual Claims per Desk Adjuster SubroScore WITH 5.7 AGENTECH Low High Potential Potential WITHOUT AGENTECHDeal of the Quarter 4455Partnership Case Study 46Partnership Case Study Loadsure and Google Cloud: Faster claims processing for shipping insurance Founded in 2018, InsurTech MGA Loadsure offers a range of dynamic freight insurance solutions that leverage data and AI to provide accurate pricing and near immediate certificate issuance. Google Cloud, offered by Google, is a suite of cloud computing services including data storage, analytics and machine learning. It offers artificial intelligence tools and services to businesses through the Vertex AI platform, and to software developers under the

In [22]:
import requests 

# Llamar a LLaMA3 local vía Ollama
def answer_with_ollama(query, k=2):
    context_chunks = search_similar(query, k)
    context = "\n\n".join(context_chunks)
    
    prompt = f"""
You are a helpful assistant answering questions based only on the following document context.

Context:
{context}

Question: {query}
Answer:
"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "mistral",
            "prompt": prompt,
            "stream": False,
            "options":{
                "temperature": 0,
                "num_predict": 300  # Máximo número de tokens de salida
        }
        }
    )

    return response.json()["response"]




In [23]:
# Ejemplo de pregunta
query = "Who is Christina Lucas?"
print(answer_with_ollama(query))

d:\CHECHO\Personal\PROYECTOS PERSONALES PORTAFOLIO\PROYECTO RAG CHATBOT GENERICO\rag_chatbot_py10\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


 Christina Lucas is the Global Head of Industry Advisory at Guidewire, a provider of cloud-based software to the Property & Casualty industry. She has an extensive career in the insurance ecosystem, having held several regional and global roles at AIG, including Head of Claims Operations for LatAm in Miami, Asia Pacific Claims Operations Officer in Tokyo, Global Head of Claims Business Performance in NYC, and leading AIG's European Center of Excellence in Bulgaria.
